# Übung 1 (Balistikbeispiel)

Die Übungsaufgaben werden als Jupyter Notebooks bereitgestellt. Ein solches Notebook besteht aus Zellen, in die Sie direkt R-Code schreiben können. Durch Klicken auf "Run" oben oder Strg+Enter wird die Zelle direkt ausgeführt und das Ergebnis wird angezeigt. Probieren Sie es in der folgenden Zelle direkt einmal aus.

In [ ]:
3 + 4
b_x, b_y = 10

ERROR: Error in parse(text = x, srcfile = src): <text>:2:4: unexpected ','
1: 3 + 4
2: b_x,
      ^


In der Vorlesung wird Bayessches Filtern anhand eines Ballistikbeispiels eingeführt.

In dieser Übung soll das Systemmodell des Ballistik-Beispiels (Kapitel 4.1 im Skript) implementiert werden. Dies werden wir später  als Grundlage verwenden, um einen Partikelfilter für dieses Beispiel zu implementieren.

Im Ballistik-Beispiel setzt sich der Zustand zu einem bestimmten Zeitpunkt aus der X- und Y-Position sowie der komponentenweisen Geschwindigkeit in X- und Y-Richtung zusammen. Zudem sind einige physikalische Konstanten gegeben. Für einen Gesamtüberblick schauen sie sich bitte Folie 4 und 5 aus dem Skript "SSuI-BIDSSS-1-Intro.pdf" an!

1.  Erstellen Sie eine Funktion `init(angle, v0, locationX)` zum Erzeugen eines Initialzustandes. Dabei soll der Abschusswinkel `angle` in Grad, die Initalgeschwindigkeit `v0` und die Abschussposition `locationX` (nur X-Koordinate, die Y-Koordinate ist immer 0, da das Projektil am Boden abgeschossen wird) übergeben werden. Definieren sie ebenfalls die physikalischen Konstanten!

In [ ]:
# plot preferences
ymin <- -100

# model preferences
time <- 100    # time [s]
tstep <- 10    # steps / sec [1/s]
times <- time*tstep
xmax <- 21000

# physical constants
g.y     <- -9.81    #m/s^2
sd.v    <- 10       #accuracy of liftoff speed [m/s]
sd.a    <- 1        #accuracy of elevation angle in [deg]
turb.x  <- 10       #acceleration by turbulence [m(s^2)]]
turb.y  <- 10       #acceleration by turbulence [m(s^2)]]
cw      <- 0.0001   #deceleration factor due to air friction [/m]
sb.o    <- 10000    # observation accuracy [m]

deg2rad <- function(x) x / 180 * pi

init <- function(angle = 45, v0 = 500, locationX = 0) {

    v       <- v0 + rnorm(1, mean=0, sd = sd.v)

    alpha   <- deg2rad(angle + rnorm(1, mean=0, sd = sd.a))
    vx      <- v * cos(alpha)
    vy      <- v * sin(alpha)

    c(locationX, 0, vx, vy)

}

2. Erstellen Sie eine Funktion `trans(x)`, die für einen Zustand `x` einen Nachfolge-Zustand berechnet. Im Skript (Kapitel 4.1) sind die Gleichungen für die Berechnung des Nachfolgezustands angegeben.

In [ ]:
trans <- function(R) {
    x   <- R[1]
    y   <- R[2]
    vx  <- R[3]
    vy  <- R[4]

    delta.t  <- 1/times

    eps.x   <- rnorm(1, mean=0, sd = turb.x)
    eps.y   <- rnorm(1, mean=0, sd = turb.y)

    ax  <-  -sign(vx * cw * vx^2 + eps.x )
    ay  <-  g.y -sign(vx * cw * vx^2 + eps.y)

    x   <- x + vx*(delta.t) + (1/2)*ax*delta.t^2
    y   <- x + vy*(delta.t) + (1/2)*ay*delta.t^2

    vx  <- vx + ax*delta.t
    vy  <- vy + ax*delta.t

    c(x, y, vx, vy)

}

3. Erstellen Sie eine Funktion `shoot(angle, v0, locationX)`, die die komplette Flugbahn des Projektils simuliert und zurückgibt (z.B. als Matrix, wobei die Zeilen die einzelnen Zeitpunkte und die Spalten die Komponenten des Zustands (x, y, vx, vy) repräsentieren.

In [ ]:
shoot <- function(angle = 45, v0 = 500, locationX = 0) {

    flight <- matrix(0, nrow=times+1, ncol=4)
    initR   <- init(angle, v0, locationX)

    flight[1,] <- initR

    for(row in 2:nrow(flight)) {

        flight[row,] = trans(flight[row-1,])

    }

    flight
}

In [ ]:
flight <- shoot(angle = 45, v0 = 500, locationX = 0)


4. Implementieren Sie eine Funktion, die eine übergebene Flugbahn plottet. Das Bild kann beispielsweise so aussehen: ![grafik.png](attachment:grafik.png)

In [ ]:
plotshot <- function(trajectory){
    #TODO
}